# RAG Summarization with BM25 and LLMs

### Implementation Description

| Step | Description | Rationale | Comment |
|------|-------------|-----------|---------|
| Process source document | Break paper into sentences. Encode the full document together, and each sentence independently. Tokenize and lemmatize text. |  | Using `verbatim_rag` to read HTML files. |
| Select relevant sentences | Calculate **BM25** similarity score between sentences and full document. Request an LLM model to extract the theme | This was the most reliable method for selecting sentences from the baseline results. |  |
| Shape summary from sentences | Use **LLM** to shape selected sentences into summary. | Plain `rank_bm25` results are disjointed and out of order. |  |
| Append supporting citation | Use `verbatim_rag` to identify supporting material in the original paper. |  |  |

#### Required Modules

In [59]:
from typing import Optional, Tuple, List

import os
import sys
import glob
import re
import json
from dotenv import load_dotenv

import pandas as pd

from IPython.display import display, Markdown

import nltk
from rank_bm25 import BM25Okapi

from verbatim_rag.schema import DocumentSchema
from verbatim_rag.chunker_providers import MarkdownChunkerProvider
from verbatim_rag.embedding_providers import SentenceTransformersProvider
from verbatim_rag.vector_stores import LocalMilvusStore
from verbatim_rag import VerbatimIndex, VerbatimRAG
from verbatim_rag.core import LLMClient

from openai import OpenAI

assert nltk.download('wordnet')
assert nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mitre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mitre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
sys.path.append(os.path.abspath("../"))

In [54]:
from utils.metrics import *

#### Constants

Using `verbatim_rag` to load documents.

In [74]:
ARXIV_URL = "https://arxiv.org/pdf/"
HTML_PATH = os.path.join("..", "data", "raw", "htmls")

documents = glob.glob(f"{HTML_PATH}/*.txt")
documents

DOCUMENT_ID = [
    '2511.21398v1',
    '2511.21444v1',
    '2511.21460v1',
    '2511.21471v1',
    '2511.21522v1',
    '2511.21569v1',
    '2511.21570v1',
    '2511.21591v1',
    '2511.21636v1',
    '2511.21678v1',
]

DOCUMENT_ID = documents[0:10]

#DOCUMENT_ID = ["..\\data\\raw\\htmls\\2510.25320v1.txt"]

#### Helper Functions

Method to extract abstract using regular expressions.

In [75]:
def abstract_from_(paper: str) -> Optional[str]:
    """Get abstract from Markdown text."""
    match = re.search(r'## Abstract\s*(.+?)(?=\n##)', paper, re.DOTALL)
    if match:
        abstract = match.group(1).strip()
        # abstract = re.sub(r"^\s*\.\s*\n*", "", abstract)
        return abstract
    return None

Method to display Markdown strings.

In [76]:
def print_markdown_(text: str) -> None:
    """Print Markdown string."""
    display(Markdown(text))

In [77]:
def parse_and_bucket_sections(full_text):
    # 1. Define the 4 Buckets
    buckets = {
        "intro": [],
        "methods": [],
        "results": [],
        "conclusion": []
    }
    
    # 2. Regex to find Top-Level Headers (e.g., "## I Introduction" or "2. Methodology")
    # Matches lines starting with numbering (I, 1, 1.) or Markdowns (##) followed by text
    # We ignore ### (subsections) effectively treating them as body text
    header_pattern = re.compile(r'^(?:##\s+|[IVX\d]+\.?\s+)(.*)', re.MULTILINE)
    
    # Split text by these headers
    # 'split' returns [text_before_first_header, header1, content1, header2, content2...]
    segments = header_pattern.split(full_text)
    
    # Iterate through pairs of (Header, Content)
    # Skip segments[0] (text before first header)
    for i in range(1, len(segments), 2):
        header_title = segments[i].strip().lower()
        content = segments[i+1].strip()
        
        # 3. Semantic Mapping Logic
        if any(x in header_title for x in ['intro', 'background', 'related', 'motivation']):
            buckets['intro'].append(content)
            
        elif any(x in header_title for x in ['method', 'formulation', 'system', 'approach', 'model', 'data', 'architecture']):
            buckets['methods'].append(content)
            
        elif any(x in header_title for x in ['result', 'experiment', 'evaluation', 'perform', 'metric', 'ablation']):
            buckets['results'].append(content)
            
        elif any(x in header_title for x in ['conclusion', 'discussion', 'future']):
            buckets['conclusion'].append(content)
            
        elif 'reference' in header_title:
            continue # Drop references
            
        else:
            # Fallback: If we can't guess, put it in Methods (safest bet for middle sections)
            # Or append to the previous bucket found
            buckets['methods'].append(content)

    # 4. Join the lists back into single text blocks
    return {
        "intro_text": "\n".join(buckets['intro']),
        "methods_text": "\n".join(buckets['methods']),
        "results_text": "\n".join(buckets['results']),
        "conclusion_text": "\n".join(buckets['conclusion'])
    }

## Process Source Documents

The `verbatim_rag` library captures the abstract correctly in all examples, unlike the processed used for the baseline models.

In [79]:
documents = {}
papers = {}
abstracts = {}
document_sections = {}

for document_id in DOCUMENT_ID:
    # paper = DocumentSchema.from_url(url=ARXIV_URL + document_id)
    #document = DocumentSchema.from_url(url=os.path.join(HTML_PATH, document_id + '.txt'))
    document = DocumentSchema.from_url(url=document_id)
    documents[document_id] = document
    papers[document_id] = document.content
    abstracts[document_id] = abstract_from_(document.content)
    document_sections[document_id] = parse_and_bucket_sections(document.content)

2026-01-12 19:49:23,844 - INFO - detected formats: [<InputFormat.HTML: 'html'>]
2026-01-12 19:49:23,961 - INFO - Going to convert document batch...
2026-01-12 19:49:23,962 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2026-01-12 19:49:23,962 - INFO - Processing document 2510.24832v1.txt
2026-01-12 19:49:24,168 - INFO - Finished converting document 2510.24832v1.txt in 0.33 sec.
2026-01-12 19:49:24,271 - INFO - detected formats: [<InputFormat.HTML: 'html'>]
2026-01-12 19:49:24,913 - INFO - Going to convert document batch...
2026-01-12 19:49:24,914 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2026-01-12 19:49:24,915 - INFO - Processing document 2510.25005v1.txt
2026-01-12 19:49:25,452 - INFO - Finished converting document 2510.25005v1.txt in 1.18 sec.
2026-01-12 19:49:25,604 - INFO - detected formats: [<InputFormat.HTML: 'html'>]
2026-01-12 19:49:25,789 - INFO - Going to convert 

## Obtain section summaries

In [139]:
def build_section_summary(client: OpenAI, prompt_template: str, sentences: str) -> str:
    response = client.responses.create(
        model=OPENAI_MODEL,
        instructions="Only reply with a summarized paragraph.",
        input=prompt_template.format(sentences=sentences)
    )
    
    return response.output_text

def build_summary(client: OpenAI, summary_wordcount: int, prompt_template: str, intro_text: str, methods_text: str, results_text: str, conclusion_text: str) -> str:
    response = client.responses.create(
        model=OPENAI_MODEL,
        instructions="""
- Start directly with the problem or the proposed solution (do not say 'This paper presents...').
- Connect the Methodology and Results logically using transition words (e.g., 'Consequently,' 'Specifically,' 'We observe that').
- Ensure the tone is objective, impersonal, and authoritative.
- Ensure the final word at least 200 relevant words long.
- Do not use bullet points. Write a single paragraph.""",
        input=prompt_template.format(summary_wordcount=summary_wordcount, 
                                     intro_text=intro_text, 
                                     methods_text=methods_text, 
                                     results_text=results_text, 
                                     conclusion_text=conclusion_text)
    )
    
    return response.output_text

def build_refiner(client: OpenAI, prompt_template: str, v1_summary: str):
    response = client.responses.create(
        model=OPENAI_MODEL,
        instructions="""
    Refinement Rules:
    - Remove generic phrases like 'comprehensive experiments show that' or 'in order to'.
    - Merge short sentences where possible to improve flow.
    - Ensure the final word count is between 150-250 words.
    - Crucial: Ensure the primary metric (the number/result) appears in the final text.""",
        input=prompt_template.format(v1_summary=v1_summary)
    )
    
    return response.output_text

def build_exam_questions(client: OpenAI, prompt_template: str, full_text: str):
    response = client.responses.create(
        model=OPENAI_MODEL,
        input=prompt_template.format(full_text=full_text)
    )
    return response.output_text
    
def build_exam_answers(client: OpenAI, prompt_template: str, summary_text: str, questions: str):
    response = client.responses.create(
        model=OPENAI_MODEL,
        input=prompt_template.format(summary_text=summary_text,
                                     questions=questions
                                    )
    )
    return response.output_text

def build_evaluation(client: OpenAI, prompt_template: str, question: str, ground_truth: str, answer: str):
    response = client.responses.create(
        model=OPENAI_MODEL,
        input=prompt_template.format(question=question,
                                     ground_truth=ground_truth, 
                                     answer=answer)
    )
    return response.output_text
    

In [252]:
INTRO_PROMPT = """
Analyze the provided Introduction. Identify the following two elements clearly:
- The Problem Space: What specific limitation, inefficiency, or challenge in the current state of the art is this paper addressing?
- The Proposed Solution: What is the core system, framework, or hypothesis introduced to solve this?

Introduction:
{sentences}
"""

METHODS_PROMPT = """
Analyze the provided Methods/Methodology. Extract the technical specifics using the following keys:
- Data/Environment: What specific datasets or source materials were used? (Do not invent any if they are not mentioned).
- The Architecture: Briefly describe the model structure, algorithm, or experimental design.
- Settings: Mention any critical hyperparameters, baselines, or comparisons used.

Methods:
{sentences}
"""

RESULTS_PROMPT = """
Analyze the provided Results. Extract the key evidence that supports the contribution:
- Quantitative Metrics: Extract the specific main performance numbers strictly as they appear in the text. If more than 2 different metrics and numbers exist, only mention the two most important metrics.
- Comparison: How did it perform relative to the baseline?
- CONSTRAINT: Only extract numbers explicitly written in the text. If no specific quantitative metrics are provided, describe the qualitative improvement (e.g., "significantly faster," "more robust") without inventing data. The numbers themselves must be an important part of the results.

Results:
{sentences}
"""

CONCLUSION_PROMPT = """
Analyze the provided Discussion/Conclusion. Summarize:
- Interpretation: Why do the results matter?
- Limitations: What is one key weakness acknowledged by the authors?
- Impact: What is the final takeaway for the field?

Conclusion:
{sentences}
"""

JOINER_PROMPT = """
You are an expert technical writer. I will provide you with the key component parts of a research paper. Your goal is to weave them into a single, fluid abstract (approx. {summary_wordcount} words).
Avoid including formulas.

Instructions:
1. Tone: Maintain an objective, scholarly tone suitable for the specific domain of the input (e.g., if the content is Computer Science, use CS terminology; if Biology, use Bio terminology).
2. Accuracy: Do not introduce new information or numbers that are not present in the Input Data.
3. Flow: Ensure smooth logical transitions between the Problem, Method, and Results.

Input Data:
- Context: {intro_text}
- Methodology: {methods_text}
- Key Findings: {results_text}
- Implications: {conclusion_text}
"""

REFINER_PROMPT = """
Review the draft abstract below.
Rewrite it to be denser and more concise, mimicking the style of a high-impact publication in this specific field.

Constraints:
- Retain all specific metrics and proper nouns (e.g., model names, dataset names).
- Do not add sensationalist adjectives (e.g., avoid "groundbreaking" unless supported by data).
- Focus on information density: reduce fluff words to allow more space for technical details.

Draft: {v1_summary}
"""

OPENAI_MODEL = 'o4-mini'

In [253]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
assert key

client = OpenAI(api_key=key)

In [254]:
rag_summaries_vs_abstract = {}
scoring_results = []

docs = document_sections.items()
#docs = [list(document_sections.items())[-1]]

for k, v in docs:
    print(f"Summarizing document: {k}")
    doc_content = document_sections[k]

    print(f"Summarizing Introduction...")
    intro_summary = build_summary_(client=client, 
                                   prompt_template=INTRO_PROMPT, 
                                   sentences=doc_content["intro_text"])
    print(f"Summarizing Methods...")
    methods_summary = build_summary_(client=client, 
                                     prompt_template=METHODS_PROMPT, 
                                     sentences=doc_content["methods_text"])
    print(f"Summarizing Results...")
    results_summary = build_summary_(client=client, 
                                     prompt_template=RESULTS_PROMPT, 
                                     sentences=doc_content["results_text"])
    print(f"Summarizing Conclusion...")
    conclusion_summary = build_summary_(client=client, 
                                        prompt_template=CONCLUSION_PROMPT, 
                                        sentences=doc_content["conclusion_text"])

    print(f"Formulating cohesive summary...")
    v1_summary = build_summary(client=client, 
                               summary_wordcount=200,
                               prompt_template=JOINER_PROMPT, 
                               intro_text=intro_summary,
                               methods_text=methods_summary, 
                               results_text=results_summary, 
                               conclusion_text=conclusion_summary
                              )

    print(f"Refining summary...")
    refined_summary = build_refiner(client=client, 
                                    prompt_template=REFINER_PROMPT, 
                                    v1_summary=v1_summary)

    curr_abstract = abstracts[k]
    rag_summaries_vs_abstract[k] = {"abstract": curr_abstract, 
                                    "v1_summary": v1_summary,
                                   "refined_summary": refined_summary}

    
    rouge_scores_subsection_llm_v1 = calculate_rouge_score(curr_abstract, 
                                                           v1_summary)
    rouge_scores_subsection_llm_refined = calculate_rouge_score(curr_abstract, 
                                                                refined_summary)
    
    bert_scores_subsection_llm_v1 = calculate_bert_score(curr_abstract, 
                                                         v1_summary)
    bert_scores_subsection_llm_refined = calculate_bert_score(curr_abstract, 
                                                              refined_summary)

    scoring_results.append(
        {
            "paper_id": k, 
            "method": "V1_SCAFFOLDED_TEMPLATING",
            "rouge1": rouge_scores_subsection_llm_v1["rouge1_fmeasure"], 
            "rougeL": rouge_scores_subsection_llm_v1["rougeL_fmeasure"],
            "bert_score_f1": bert_scores_subsection_llm_v1["bertscore_f1"]
        }
    )
    
    scoring_results.append(
            {
                "paper_id": k, 
                "method": "REFINED_SCAFFOLDED_TEMPLATING",
                "rouge1": rouge_scores_subsection_llm_refined["rouge1_fmeasure"], 
                "rougeL": rouge_scores_subsection_llm_refined["rougeL_fmeasure"],
                "bert_score_f1": bert_scores_subsection_llm_refined["bertscore_f1"]
            }
        )

df_results = pd.DataFrame(scoring_results)
df_results

Summarizing document: ..\data\raw\htmls\2510.24832v1.txt
Summarizing Introduction...


2026-01-12 23:15:23,403 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Summarizing Methods...


2026-01-12 23:15:33,853 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Summarizing Results...


...

Formulating cohesive summary...


2026-01-12 23:26:06,220 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Refining summary...


2026-01-12 23:26:26,389 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2026-01-12 23:26:26,392 - INFO - Using default tokenizer.
2026-01-12 23:26:26,414 - INFO - Using default tokenizer.


,paper_id,method,rouge1,rougeL,bert_score_f1
0,..\data\raw\htmls\2510.24832v1.txt,V1_SCAFFOLDED_TEMPLATING,0.480000,0.224000,0.629264
1,..\data\raw\htmls\2510.24832v1.txt,REFINED_SCAFFOLDED_TEMPLATING,0.458128,0.216749,0.624795
2,..\data\raw\htmls\2510.25005v1.txt,V1_SCAFFOLDED_TEMPLATING,0.260274,0.164384,0.583295
3,..\data\raw\htmls\2510.25005v1.txt,REFINED_SCAFFOLDED_TEMPLATING,0.303030,0.199134,0.588976
4,..\data\raw\htmls\2510.25007v1.txt,V1_SCAFFOLDED_TEMPLATING,0.262425,0.147117,0.552201
5,..\data\raw\htmls\2510.25007v1.txt,REFINED_SCAFFOLDED_TEMPLATING,0.274882,0.146919,0.549716
6,..\data\raw\htmls\2510.25014v1.txt,V1_SCAFFOLDED_TEMPLATING,0.416667,0.200617,0.641713
7,..\data\raw\htmls\2510.25014v1.txt,REFINED_SCAFFOLDED_TEMPLATING,0.434004,0.241611,0.649669
8,..\data\raw\htmls\2510.25065v1.txt,V1_SCAFFOLDED_TEMPLATING,0.375000,0.185185,0.607236
9,..\data\raw\htmls\2510.25065v1.txt,REFINED_SCAFFOLDED_TEMPLATING,0.429752,0.214876,0.611708


In [255]:
[(len(d["abstract"]), len(d["v1_summary"]), len(d["refined_summary"])) for d in rag_summaries_vs_abstract.values()]

[(1341, 1961, 1293),
 (453, 1659, 1199),
 (1356, 2078, 1439),
 (1380, 3069, 1623),
 (1116, 1934, 1469),
 (2025, 2054, 1496),
 (1601, 2245, 1442),
 (1294, 2149, 1382),
 (2025, 2741, 1672),
 (1809, 2311, 1522)]

In [256]:
with open("./summary_templating.json", "w") as f:
    json.dump(rag_summaries_vs_abstract, f)

In [265]:
EXAMINER_PROMPT = """
You are an expert researcher. Read the following Full Paper text.
Generate 5 fact based questions that must be simple enough to be answered from a theoretical scientific abstract for this paper. Keep the answers limited to a single sentence if possible. 
You must ignore the paper's Abstract section and instead only rely on the rest of the paper. Choose your own questions to be answered that you consider would create a high quality scientific abstract based on the Introduction, Methodology, Results and Conclusions.
For each question, provide the exact Ground Truth Answer from the text.

Constraints:
1. Question 1 must be about the **specific problem** addressed.
2. Question 2 must be about the **methodology** used.
3. Question 3 must be about the **quantitative results** (metrics, numbers).
4. Question 4 must be about the **dataset or environment**.
5. Question 5 must be about the **limitations or future work**.

Format output as python compatible JSON list (in text format) with a question property and an answer property for each of the questions.

Full Paper Text: {full_text}
"""

STUDENT_PROMPT = """
You are a student taking a test.
Read the following Summary and answer the Questions based *strictly* on the information provided in that summary.
If the information is not in the summary, answer "NOT MENTIONED".

Summary: {summary_text}

Questions:
{questions}

Format the output as a python list of answers.
"""

GRADER_PROMPT = """
Compare the Student's Answer to the Ground Truth Answer.
Score 1 if the Student's Answer is correct and sufficiently semantically similar to the Ground Truth (typos and incomplete answers are OK, but complete fabrications are not acceptable).
Score 0 if the Student's Answer is incorrect, vague, or "NOT MENTIONED".

Q: {question}
Ground Truth: {ground_truth}
Student Answer: {answer}

Return ONLY the score (0 or 1).
"""

In [266]:
results = []

for paper_id, paper_content in papers.items():
    print("#######", paper_id)
    # 1. Generate the Exam (using Full Text)
    exam_json = build_exam_questions(client=client, 
                                     prompt_template=EXAMINER_PROMPT, 
                                     #full_text=paper_content.replace())
                                     full_text="\n".join([v for v in document_sections[paper_id].values()])
                                    )
    questions = json.loads(exam_json)
    #print(questions)
    
    # Test summary answers
    summary_answers = build_exam_answers(client=client, 
                                         prompt_template=STUDENT_PROMPT, 
                                         summary_text=rag_summaries_vs_abstract[paper_id]["v1_summary"], 
                                         questions=questions)
    summary_answers = json.loads(summary_answers)

    # Test summary answers
    refined_summary_answers = build_exam_answers(client=client, 
                                         prompt_template=STUDENT_PROMPT, 
                                         summary_text=rag_summaries_vs_abstract[paper_id]["refined_summary"], 
                                         questions=questions)
    refined_summary_answers = json.loads(refined_summary_answers)
    
    # Test abstract answers
    baseline_answers = build_exam_answers(client=client, 
                                          prompt_template=STUDENT_PROMPT, 
                                          summary_text=rag_summaries_vs_abstract[paper_id]["abstract"], 
                                          questions=questions)
    baseline_answers = json.loads(baseline_answers)
    
    # 4. Grading
    summary_score = 0
    refined_summary_score = 0
    abstract_score = 0
    
    for i, q in enumerate(questions):
        print("\n")
        print(f"Question: {q['question']}")
        print(f"Ground truth: {q['answer']}")
        
        summary_grade = build_evaluation(client=client,
                                         prompt_template=GRADER_PROMPT, 
                                        question=q['question'], 
                                        ground_truth=q['answer'], 
                                        answer=summary_answers[i])
        print(f"Summary answer: {summary_answers[i]}")
        if "1" in summary_grade: 
            summary_score += 1

        refined_summary_grade = build_evaluation(client=client,
                                         prompt_template=GRADER_PROMPT, 
                                        question=q['question'], 
                                        ground_truth=q['answer'], 
                                        answer=refined_summary_answers[i])
        print(f"Refined summary answer: {refined_summary_answers[i]}")
        if "1" in refined_summary_grade: 
            refined_summary_score += 1
        
        abstract_grade = build_evaluation(client=client,
                                          prompt_template=GRADER_PROMPT, 
                                        question=q['question'], 
                                        ground_truth=q['answer'], 
                                        answer=baseline_answers[i])
        print(f"Abstract answer: {baseline_answers[i]}")
        if "1" in abstract_grade: 
            abstract_score += 1

    results.append({
        "paper_id": paper_id,
        "summary_recall": summary_score / 5.0,
        "refined_summary_recall": summary_score / 5.0,
        "abstract_recall": abstract_score / 5.0
    })
    print()
    
# Calculate Average Improvement
avg_summary_recall = sum(r['summary_recall'] for r in results) / len(results)
avg_refined_summary_recall = sum(r['refined_summary_recall'] for r in results) / len(results)
avg_abstract_recall = sum(r['abstract_recall'] for r in results) / len(results)

print(f"RAG Summary Recall: {avg_summary_recall:.2%}")
print(f"RAG Refined Summary Recall: {avg_refined_summary_recall:.2%}")
print(f"Original Abstract Recall: {avg_abstract_recall:.2%}")

####### ..\data\raw\htmls\2510.24832v1.txt


2026-01-12 23:35:36,361 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2026-01-12 23:36:02,440 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2026-01-12 23:36:14,849 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2026-01-12 23:36:23,453 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"




Question: What specific problem in RLVR data scheduling for LLM mathematical reasoning does this paper address?
Ground truth: Existing RLVR data scheduling methods estimate query difficulty primarily via final solution accuracy and overlook richer query-level characteristics such as the structural complexity of the reasoning tree.


2026-01-12 23:36:25,158 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Summary answer: Existing RLVR data-scheduling methods rely on final solution accuracy as a proxy for problem difficulty and overlook the structural complexity of a query’s reasoning tree, leading to misprioritization of examples.


...

Summary answer: Enhancing controllability, scalability, and cross-domain applicability.


2026-01-12 23:48:43,124 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Refined summary answer: NOT MENTIONED


2026-01-12 23:48:44,527 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Abstract answer: NOT MENTIONED
RAG Summary Recall: 60.00%
RAG Refined Summary Recall: 60.00%
Original Abstract Recall: 36.00%
